This notebook includes per fill calculations with BCM data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.dates as dates
import seaborn as sns, paramiko, os, re, tables, glob, sys, shutil
from matplotlib import pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib.patches import Rectangle

from matplotlib import gridspec
from ipywidgets import interact, IntSlider
if "./src" not in sys.path:
    sys.path.insert(0, "./src")
from src import bcm_utils
from sklearn.linear_model import LinearRegression
from scipy.optimize import curve_fit
import seaborn as sns
import pylab

# Define parameters

brilcalc command: <br>
brilcalc lumi --normtag /cvmfs/cms-bril.cern.ch/cms-lumi-pog/Normtags/normtag_PHYSICS.json -u 1e30/cm2s -o example.csv --begin "04/01/18 00:00:00" --end "05/01/18 00:00:00" --byls

1 𝜇b -1 s -1 = 10 30 cm −2 s −1

Connect to remote: sshfs /path/to/remote/folder /path/to/local/mount

In [ ]:
"""
lumi
"""
lumi_threshold = 0

"""
settings
"""
save_plots = False
do_fit = True

#paths
data_path = "/Users/pkicsiny/sshfs/json_pickles"
brilcalc_dir = "../brilcalc_data/2018_offline.csv"

year = 2018
months = [str(m).zfill(2) for m in list(range(4,11))]
rs = 10
colnames = bcm_utils.get_column_names([24, 48],[rs])

outlier_fills = {
    "04": [6612, 6613, 6614, 6615, 6621, 6628],
    "05": [6628, 6629, 6633, 6636, 6640, 6641, 6688, 6690, 6706, 6747],
    "06": [6854, 6858],
    "07": [6901, 6911, 6919, 6929, 6939],
    "08": [7006, 7036, 7043],
    "09": [7213],
    "10": [],
}

#plot settings
outlier_thr_smoothing = 5
moving_avg_window = 500
x_min = -100
x_max = 31000
fit_min = lumi_threshold + 0
fit_max = 25000
y_lim = 1*(5-lumi_threshold*1e-3 + 1.5)
unit = r"$10^{30}$cm$^{-2}$s$^{-1}$" #r"$\mathrm{\mu}$bs$^{-1}"
unit_label = r"$10^{-30}$$\mathrm{\mu}$A$\,$cm$^{2}$s" #"A/{}".format(unit)

def func(x, a):
    return a*x

In [ ]:
#Read monthly json pickles
data_df = pd.DataFrame()
for month in months:
    print("Reading month {}".format(month))
    blm_current = pd.read_pickle(data_path+"/pickle_{}".format(month))/bcm_utils.γ[9]/bcm_utils.β
    blm_current.index = [pd.Timestamp(ts).tz_localize(None) for ts in blm_current.index]
    
    # composite offline normtag
    lumi_dir = "../brilcalc_data/inst_{}.csv".format(month)

    #Read bunch by bunch lumi from brilcalc
    lumi_df = pd.read_csv(lumi_dir, header=1)[:-3]
    lumi_df["fill"] = lumi_df["#run:fill"].str.split(':', 1).str[1]
    inst_lumi = pd.DataFrame(lumi_df[["delivered(1e30/cm2s)", "fill"]]).astype(float)
    inst_lumi.index = pd.to_datetime(lumi_df["time"])
    inst_lumi = inst_lumi[inst_lumi["delivered(1e30/cm2s)"] > lumi_threshold]
    data_df = pd.concat([data_df, pd.merge(blm_current, inst_lumi, left_index=True, right_index=True)])
    print("Size of data df: {}".format(len(data_df)))
    
#calculate ratios
data_df["ratios_-Z"] = data_df["CH24RS9"].divide(data_df["delivered(1e30/cm2s)"])
data_df["ratios_+Z"] = data_df["CH48RS9"].divide(data_df["delivered(1e30/cm2s)"])

In [ ]:
data_df["delivered(1e30/cm2s)"].plot() # 1e4
data_df[data_df["fill"] == 6770]["delivered(1e30/cm2s)"].plot()

In [ ]:
# 1e7 but per lumisection (23s) average instantaneous-lumi-per-sec -> order 1e8 1e30 cm-2 = 1e14 b-1 
plt.scatter(data_df["fill"].unique(), 
            [data_df[data_df["fill"] == f]["delivered(1e30/cm2s)"].sum()*23 for f in data_df["fill"].unique()], 
        label="int. lumi per fill")
plt.axhline(1e8, c="r", label="lumi threshold")
plt.legend()

In [ ]:
X_far = X[np.abs(diff) > 0*np.std(diff)]
outliers = X_Y[X_Y.index.isin(X_far.index)]

for fill in outliers.fill.unique()[:100]:
    print(fill)
    fig, ax = plt.subplots(figsize=(8, 8))
    
    fill_data = outliers[outliers["fill"] == fill]
    norm = fill_data["delivered(1e30/cm2s)"].max()/fill_data["CH24RS9"].max()
    ax.plot(fill_data.sort_index().index, norm*fill_data.sort_index()["CH24RS9"],
                          c="b",
                          label="BLM current")
    ax.set_xlabel("Time [UTC]",  fontsize=14)
    ax.set_ylabel("BLM current [A]", fontsize=14)
    ax.set_title("Fill {}".format(fill), fontsize=18)
    
    ax.scatter(fill_data.sort_index().index, fill_data.sort_index()["delivered(1e30/cm2s)"],
                  c="r", s=4, label="Inst. lumi")
    ax.set_xlabel("Time [UTC]",  fontsize=14)
    ax.set_ylabel("Arbitrary unit", fontsize=14)
    ax.grid()
    ax.legend()
    ax.set_title("Fill {}".format(int(fill)), fontsize=18)
    plt.savefig("plots/outlier_fills/fill_{}.png".format(int(fill)))

In [ ]:
"""
negative side
"""

X_Y = data_df.sort_values(by="delivered(1e30/cm2s)", ascending=True)      
X = X_Y["delivered(1e30/cm2s)"]
Y = X_Y["CH24RS9"].astype(float)

Y_mua = 1e6*Y

nx = 2
ny = 5
n_bins = nx*ny
x_bins = np.linspace(X.min(), X.max(), 11)

fig, ax = plt.subplots(nx, ny, figsize=(35,14))

bin_center_list = []
mean_list = []
std_list = []
muerror_list = []

for i in range(n_bins):
    
    #select elements that belong to current bin
    X_bin = X[(X >= x_bins[i]) & (X < x_bins[i+1])]
    Y_bin = Y_mua[(X >= x_bins[i]) & (X < x_bins[i+1])]
    
    print(len(Y_bin))
    H = sum(Y_bin)
    E = sum(Y_bin**2)
    h = H/len(Y_bin)
    s = np.sqrt(E/len(Y_bin) - h**2)
    e = s/np.sqrt(len(Y_bin))

    bin_center_list.append(X_bin.mean())
    mean_list.append(h)
    std_list.append(s)
    muerror_list.append(e)
    
    idx = 0 if i < ny else 1
    ax[idx, i%ny].hist(Y_bin, label=f"Inst. luminosity [%s] $\in$ [%s, %s]\n$\mu$=%.3E\n$\sigma$=%.2E"%
                      (unit, round(X_bin[0]), round(X_bin[-1]), h, s))
    ax[idx, i%ny].set_yscale("log")
    ax[idx, i%ny].set_ylim(0,1e6)
    ax[idx, i%ny].grid()
    ax[idx, i%ny].legend()
    ax[idx, i%ny].set_xlabel("BLM -Z current [$\mathrm{\mu}$A]", fontsize=16)
    ax[idx, i%ny].set_ylabel("Count", fontsize=16)
    
plt.savefig("plots/negative_nonlinearity_{}_histos.png".format(lumi_threshold, year))

plt.figure(figsize=(9, 6))
pylab.errorbar(bin_center_list, mean_list, yerr=std_list, marker='',ls='',zorder=0, capsize=3, c="k")
pylab.scatter(bin_center_list, mean_list, c="k")
plt.grid()
constrained_fit, constrained_fit_cov = curve_fit(func, bin_center_list, mean_list)
std_dev_fit_param = np.sqrt(constrained_fit_cov)[0][0]
std_dev_percent = std_dev_fit_param/constrained_fit[0]*100
pylab.plot(bin_center_list, func(np.array(bin_center_list), *constrained_fit), c="b",
           label=r"Fit: y = a * x"+"\n"+
             "a = %.5E [%s]"%(constrained_fit[0], unit_label)+"\n"+
            r"$\Delta$a = %.2E [%s]" %
             (std_dev_fit_param, unit_label), linewidth=2)
pylab.legend()
pylab.legend(fontsize=14)
pylab.ylabel(r"BLM -Z current [$\mathrm{\mu}$A]", fontsize=16)
pylab.title(r'$\bf{CMS}$ Preliminary'+"            "+r'Offline Luminosity 2018 ($\sigma_{\mathrm{inel}}$=79.5 mb, $\sqrt{\mathrm{s}}$=13 TeV)',
             loc='right', fontsize=13)
pylab.xlabel(r"Inst. luminosity [{}]".format(unit), fontsize=16)
#pylab.gca().tick_params(axis='both', which='major', labelsize=16)
#pylab.gca().tick_params(axis='both', which='major', labelsize=16)
#pylab.text(15000, 0.05*(1.05*Y.max()+1e-8), r'$\bf{CMS}$ $\it{Preliminary}$', size=20)
plt.savefig("plots/lumi_th_{}/negative_nonlinearity_{}_new.png".format(lumi_threshold, year))

In [ ]:
"""
positive side
"""

X_Y = data_df.sort_values(by="delivered(1e30/cm2s)", ascending=True)      
X = X_Y["delivered(1e30/cm2s)"]
Y = X_Y["CH48RS9"].astype(float)

Y_mua = 1e6*Y

nx = 2
ny = 5
n_bins = nx*ny
x_bins = np.linspace(X.min(), X.max(), 11)

fig, ax = plt.subplots(nx, ny, figsize=(35,14))

bin_center_list = []
mean_list = []
std_list = []
muerror_list = []

for i in range(n_bins):
    
    #select elements that belong to current bin
    X_bin = X[(X >= x_bins[i]) & (X < x_bins[i+1])]
    Y_bin = Y_mua[(X >= x_bins[i]) & (X < x_bins[i+1])]
    
    print(len(Y_bin))
    H = sum(Y_bin)
    E = sum(Y_bin**2)
    h = H/len(Y_bin)
    s = np.sqrt(E/len(Y_bin) - h**2)
    e = s/np.sqrt(len(Y_bin))

    bin_center_list.append(X_bin.mean())
    mean_list.append(h)
    std_list.append(s)
    muerror_list.append(e)
    
    idx = 0 if i < ny else 1
    ax[idx, i%ny].hist(Y_bin, label=f"Inst. luminosity [%s] $\in$ [%s, %s]\n$\mu$=%.3E\n$\sigma$=%.2E"%
                      (unit, round(X_bin[0]), round(X_bin[-1]), h, s))
    ax[idx, i%ny].set_yscale("log")
    ax[idx, i%ny].set_ylim(0,1e6)
    ax[idx, i%ny].grid()
    ax[idx, i%ny].legend()
    ax[idx, i%ny].set_xlabel("BLM +Z current [$\mathrm{\mu}$A]", fontsize=16)
    ax[idx, i%ny].set_ylabel("Count", fontsize=16)
    
plt.savefig("plots/positive_nonlinearity_{}_histos.png".format(lumi_threshold, year))

plt.figure(figsize=(9, 6))
pylab.errorbar(bin_center_list, mean_list, yerr=std_list, marker='',ls='',zorder=0, capsize=3, c="k")
pylab.scatter(bin_center_list, mean_list, c="k")
plt.grid()
constrained_fit, constrained_fit_cov = curve_fit(func, bin_center_list, mean_list)
std_dev_fit_param = np.sqrt(constrained_fit_cov)[0][0]
std_dev_percent = std_dev_fit_param/constrained_fit[0]*100
pylab.plot(bin_center_list, func(np.array(bin_center_list), *constrained_fit), c="b",
           label=r"Fit: y = a * x"+"\n"+
             "a = %.5E [%s]"%(constrained_fit[0], unit_label)+"\n"+
            r"$\Delta$a = %.2E [%s]" %
             (std_dev_fit_param, unit_label), linewidth=2)
pylab.legend()
pylab.legend(fontsize=14)
pylab.ylabel(r"BLM +Z current [$\mathrm{\mu}$A]", fontsize=16)
pylab.title(r'$\bf{CMS}$ Preliminary'+"            "+r'Offline Luminosity 2018 ($\sigma_{\mathrm{inel}}$=79.5 mb, $\sqrt{\mathrm{s}}$=13 TeV)',
             loc='right', fontsize=13)
pylab.xlabel(r"Inst. luminosity [{}]".format(unit), fontsize=16)
#pylab.gca().tick_params(axis='both', which='major', labelsize=16)
#pylab.gca().tick_params(axis='both', which='major', labelsize=16)
#pylab.text(15000, 0.05*(1.05*Y.max()+1e-8), r'$\bf{CMS}$ $\it{Preliminary}$', size=20)
plt.savefig("plots/lumi_th_{}/positive_nonlinearity_{}_new.png".format(lumi_threshold, year))

In [ ]:
plt.figure(figsize=(10,10))
sns.regplot(x=X, y=Y, x_bins=10, fit_reg=False)

In [ ]:
"""
negative side
"""
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1], bottom=.261, hspace=0)

#plot
X_Y = data_df.sort_values(by="delivered(1e30/cm2s)", ascending=True)      
X = X_Y["delivered(1e30/cm2s)"]
Y = X_Y["CH24RS9"].astype(float)
ax1.scatter(X, Y, s=1, c="k", label="No. of data points: {}".format(len(X)))
ax1.grid()
ax1.legend(fontsize=18)
ax1.set_ylabel("BLM current [A]", fontsize=20)#, horizontalalignment='left')
ax1.set_xticklabels([])#r"Inst. luminosity [$10^{30}$cm$^{-2}$s$^{-1}$]", fontsize=20, horizontalalignment='right', x=1.0)
ax1.set_xlim(x_min, x_max)
ax1.get_yaxis().set_label_coords(-0.045,0.62)
ax1.set_title(r'$\bf{{{CMS}}}$ Offline Luminosity 2018 ($\sigma_{\mathrm{inel}}$=71.3 mb, $\sqrt{\mathrm{s}}$=13 TeV)', loc='right', fontsize=20)

if do_fit:
    
    #fit line
    X_low = X[(X<fit_max) & (X>fit_min)]
    Y_low = Y[X_low.index.values]
    print("Fitting line on samples between {}-{}".format(fit_min, fit_max))
    constrained_fit, constrained_fit_cov = curve_fit(func, X_low, Y_low)
    std_dev_fit_param = np.sqrt(constrained_fit_cov)[0][0]
    std_dev_percent = std_dev_fit_param/constrained_fit[0]*100
    ax1.plot(X, func(X, *constrained_fit), c="b",
             label=r"Fit: y = a * x"+"\n"+
             "a = %.6E [%s]"%(constrained_fit[0], unit_label)+"\n"+
            r"$\Delta$a = %.2E [%s]" %
             (std_dev_fit_param, unit_label), linewidth=2)
    
    rect = Rectangle((0, -outlier_thr_smoothing*np.std(Y - func(X, *constrained_fit))),
                     X.max(), 2*outlier_thr_smoothing*np.std(Y - func(X, *constrained_fit)),
                     angle=np.degrees(np.arctan2(constrained_fit[0], 1)), linewidth=1,
                     edgecolor='lightblue',facecolor='lightblue', alpha=.5,
                     zorder=0, label="$\pm %s\sigma$ deviation from fit"%(outlier_thr_smoothing))
    ax1.add_patch(rect)
    mask = Rectangle((0, 0), X.max(),Y.max(), facecolor="none", edgecolor="none")
    ax1.add_patch(mask)
    rect.set_clip_path(mask)
    ax1.set_ylim(bottom=-1e-8, top=1.05*Y.max())

    #plot deviations
    ax2 = plt.subplot(gs[1])
    
    diff = Y - func(X, *constrained_fit)
    X_close = X[np.abs(diff) < outlier_thr_smoothing*np.std(diff)]
    Y_close = Y[np.abs(diff) < outlier_thr_smoothing*np.std(diff)]
    ax1.scatter(X_close, Y_close, c="green", s=1)
    
    rel_diff = 100*(Y_close-func(X_close, *constrained_fit))/func(X_close, *constrained_fit)
    rel_diff_smooth = rel_diff.rolling(window=moving_avg_window).mean()
    
    dev_1 = 100*(Y - func(X, *constrained_fit))/func(X, *constrained_fit)
    dev_1 = dev_1[np.abs(dev_1-dev_1.mean()) < outlier_thr_smoothing*dev_1.std()]
    dev_1_smooth = dev_1.rolling(window=moving_avg_window).mean()
    ax2.plot(X_close, np.abs(rel_diff_smooth),
             label=f"Rel. abs. error of samples\nwithin %s$\sigma$ from fit\nMoving avg. window\nlength: %s"%
             (outlier_thr_smoothing, moving_avg_window)+r" [{}]".format(unit), c="b")
    ax2.text(25000, 1e-5, r'$\bf{CMS}$ $\it{Preliminary}$', size=20)
    ax2.hlines(1, X_close.min(), X_close.max(), label="1$\%$")
    #rect = Rectangle((0,-1),X.max(),2, linewidth=1,
    #            edgecolor='lightblue',facecolor='lightblue', alpha=.8, zorder=0, label="$\pm 1\%$")
    #ax2.add_patch(rect)    
    ax2.grid()
    ax2.set_yscale("log")
    ax2.legend(loc="upper right", fontsize=18)
    ax2.set_ylabel("Rel. abs. error [%]", fontsize=20, horizontalalignment='left')
    ax2.set_xlabel(r"Inst. luminosity [{}]".format(unit), fontsize=20)#, horizontalalignment='right', x=1.0)
    ax2.set_xlim(x_min, x_max)
    #ax2.set_ylim(-3, 22)
    ax2.get_yaxis().set_label_coords(-0.045,0.06)
    
    ax1.legend(fontsize=18)

fig.tight_layout()
ax1.tick_params(axis='both', which='major', labelsize=16)
ax2.tick_params(axis='both', which='major', labelsize=16)

ax1.text(25000, 0.05*(1.05*Y.max()+1e-8), r'$\bf{CMS}$ $\it{Preliminary}$', size=20)

print(std_dev_percent)
if save_plots:
    plt.savefig("plots/lumi_th_{}/negative_nonlinearity_{}.png".format(lumi_threshold, year))

# plot corrected data stream
#W = poly1d_fn(X)/poly2d_fn(X)
#ax1.scatter(X, Y*W, s=1, c="g", label="No. of data points: {}".format(len(X)))

In [ ]:
"""
positive side
"""
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 12))
gs = gridspec.GridSpec(2, 1, height_ratios=[2, 1], bottom=.261, hspace=0)

#plot
X_Y = data_df.sort_values(by="delivered(1e30/cm2s)", ascending=True)      
X = X_Y["delivered(1e30/cm2s)"]
Y = X_Y["CH48RS9"].astype(float)
ax1.scatter(X, Y, s=3, c="k", label="No. of data points: {}".format(len(X)))
ax1.grid()
ax1.legend(fontsize=18)
ax1.set_ylabel("BLM current [A]", fontsize=20)#, horizontalalignment='left')
ax1.set_xticklabels([])#r"Inst. luminosity [$10^{30}$cm$^{-2}$s$^{-1}$]", fontsize=20, horizontalalignment='right', x=1.0)
ax1.set_xlim(x_min, x_max)
ax1.get_yaxis().set_label_coords(-0.045,0.62)
ax1.set_title(r'$\bf{{{CMS}}}$ Offline Luminosity 2018 ($\sigma_{\mathrm{inel}}$=71.3 mb, $\sqrt{\mathrm{s}}$=13 TeV)', loc='right', fontsize=20)

if do_fit:
    
    #fit line
    X_low = X[(X<fit_max) & (X>fit_min)]
    Y_low = Y[X_low.index.values]
    print("Fitting line on samples between {}-{}".format(fit_min, fit_max))
    constrained_fit, constrained_fit_cov = curve_fit(func, X_low, Y_low)
    std_dev_fit_param = np.sqrt(constrained_fit_cov)[0][0]
    std_dev_percent = std_dev_fit_param/constrained_fit[0]*100
    ax1.plot(X, func(X, *constrained_fit), c="b",
             label=r"Fit: y = a * x"+"\n"+
             "a = %.6E [%s]"%(constrained_fit[0], unit_label)+"\n"+
            r"$\Delta$a = %.2E [%s]" %
             (std_dev_fit_param, unit_label), linewidth=2)
    
    rect = Rectangle((0, -outlier_thr_smoothing*np.std(Y - func(X, *constrained_fit))),
                     X.max(), 2*outlier_thr_smoothing*np.std(Y - func(X, *constrained_fit)),
                     angle=np.degrees(np.arctan2(constrained_fit[0], 1)), linewidth=1,
                     edgecolor='lightblue',facecolor='lightblue', alpha=.8,
                     zorder=0, label="$\pm %s\sigma$ deviation from fit"%(outlier_thr_smoothing))
    ax1.add_patch(rect)
    mask = Rectangle((0, 0), X.max(),Y.max(), facecolor="none", edgecolor="none")
    ax1.add_patch(mask)
    rect.set_clip_path(mask)
    ax1.set_ylim(bottom=-1e-8, top=1.05*Y.max())

    #plot deviations
    ax2 = plt.subplot(gs[1])
    
    diff = Y - func(X, *constrained_fit)
    X_close = X[np.abs(diff) < outlier_thr_smoothing*np.std(diff)]
    Y_close = Y[np.abs(diff) < outlier_thr_smoothing*np.std(diff)]
    ax1.scatter(X_close, Y_close, c="green", s=3)
    
    rel_diff = 100*(Y_close-func(X_close, *constrained_fit))/func(X_close, *constrained_fit)
    rel_diff_smooth = rel_diff.rolling(window=moving_avg_window).mean()
    
    dev_1 = 100*(Y - func(X, *constrained_fit))/func(X, *constrained_fit)
    dev_1 = dev_1[np.abs(dev_1-dev_1.mean()) < outlier_thr_smoothing*dev_1.std()]
    dev_1_smooth = dev_1.rolling(window=moving_avg_window).mean()
    ax2.plot(X_close, np.abs(rel_diff_smooth),
             label=f"Rel. abs. error of samples\nwithin %s$\sigma$ from fit\nMoving avg. window\nlength: %s"%
             (outlier_thr_smoothing, moving_avg_window)+r" [{}]".format(unit), c="b")
    ax2.text(25000, 1e-5, r'$\bf{CMS}$ $\it{Preliminary}$', size=20)
    ax2.hlines(1, X_close.min(), X_close.max(), label="1$\%$")
    #rect = Rectangle((0,-1),X.max(),2, linewidth=1,
    #            edgecolor='lightblue',facecolor='lightblue', alpha=.8, zorder=0, label="$\pm 1\%$")
    #ax2.add_patch(rect)    
    ax2.grid()
    ax2.set_yscale("log")
    ax2.legend(loc="upper right", fontsize=18)
    ax2.set_ylabel("Rel. abs. error [%]", fontsize=20, horizontalalignment='left')
    ax2.set_xlabel(r"Inst. luminosity [{}]".format(unit), fontsize=20)#, horizontalalignment='right', x=1.0)
    ax2.set_xlim(x_min, x_max)
    #ax2.set_ylim(-3, 22)
    ax2.get_yaxis().set_label_coords(-0.045,0.06)
    
    ax1.legend(fontsize=18)

fig.tight_layout()
ax1.tick_params(axis='both', which='major', labelsize=16)
ax2.tick_params(axis='both', which='major', labelsize=16)

ax1.text(25000, 0.05*(1.05*Y.max()+1e-8), r'$\bf{CMS}$ $\it{Preliminary}$', size=20)

print(std_dev_percent)
if save_plots:
    plt.savefig("plots/lumi_th_{}/positive_nonlinearity_{}.png".format(lumi_threshold, year))

# plot corrected data stream
#W = poly1d_fn(X)/poly2d_fn(X)
#ax1.scatter(X, Y*W, s=1, c="g", label="No. of data points: {}".format(len(X)))

In [ ]:
X_close = X[np.abs(Y - func(X, *constrained_fit)) < outlier_thr_smoothing*np.std(Y - func(X, *constrained_fit))]
Y_close = Y[np.abs(Y - func(X, *constrained_fit)) < outlier_thr_smoothing*np.std(Y - func(X, *constrained_fit))]

plt.plot(X_close, Y_close - func(X_close, *constrained_fit))

In [ ]:
plt.plot(X, Y - func(X, *constrained_fit))
plt.hlines(3*np.std(Y - func(X, *constrained_fit)), 0, X.max(), colors="r", zorder=11111111)
plt.hlines(-3*np.std(Y - func(X, *constrained_fit)), 0, X.max(), colors="r", zorder=11111111)
plt.plot(X_close, Y_close - func(X_close, *constrained_fit), c="green")

# Statistics of fit parameters

In [ ]:
stats_0 = pd.DataFrame(index=months,
                     columns=[["negative", "negative", "positive", "positive"],
                              ["a", f"$\Delta$a", "a", f"$\Delta$a"]])
stats_0["negative", "a"] = pd.Series(
    [3.49, 3.49, 3.48, 3.52, 3.49, 3.51, 3.48], index=months)*1e-11
stats_0["negative", f"$\Delta$a"] = pd.Series(
    [1.65, 6.29, 6, 4.94, 3.51, 10.1, 5.12], index=months)*1e-14
stats_0["positive", "a"] = pd.Series(
    [1.98, 1.99, 1.98, 2, 1.99, 2, 1.99], index=months)*1e-11
stats_0["positive", f"$\Delta$a"] = pd.Series(
    [0.94, 3.58, 3.41, 2.81, 2, 5.77, 2.92], index=months)*1e-14
stats_0

In [ ]:
stats_5000 = pd.DataFrame(index=months,
                     columns=[["negative", "negative", "positive", "positive"],
                              ["a", f"$\Delta$a", "a", f"$\Delta$a"]])
stats_5000["negative", "a"] = pd.Series(
    [3.48, 3.47, 3.46, 3.5, 3.48, 3.48, 3.47], index=months)*1e-11
stats_5000["negative", f"$\Delta$a"] = pd.Series(
    [1.88, 4.23, 5.68, 8.03, 2.46, 8.48, 3.36], index=months)*1e-14
stats_5000["positive", "a"] = pd.Series(
    [1.98, 1.98, 1.97, 1.99, 1.98, 1.99, 1.98], index=months)*1e-11
stats_5000["positive", f"$\Delta$a"] = pd.Series(
    [1.07, 2.41, 3.23, 4.57, 1.4, 4.83, 1.92], index=months)*1e-14
stats_5000